In [48]:
EXCHANGE_LIST = ['nyse', 'nasdaq', 'amex']
MARKET_CAP_MIN = 10000  # in millions

# constants
COUNTRY_USA = 'United States'
ONE_MILLION = 1000000

In [49]:
import pandas as pd 
from enum import Enum
import io
import requests
import time

In [50]:
# ref: https://github.com/shilewenuw/get_all_tickers

# headers and params used to bypass NASDAQ's anti-scraping mechanism in function __exchange2df
nasdaq_headers = {
    'authority': 'api.nasdaq.com',
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'origin': 'https://www.nasdaq.com',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.nasdaq.com/',
    'accept-language': 'en-US,en;q=0.9',
}

def nasdaq_params(exchange):
    return (
        ('letter', '0'),
        ('exchange', exchange),
        ('download', 'true'),
    )

def exchange_to_df(exchange):
    r = requests.get('https://api.nasdaq.com/api/screener/stocks', \
                     headers=nasdaq_headers, params=nasdaq_params(exchange))
    data = r.json()['data']
    df = pd.DataFrame(data['rows'], columns=data['headers'])
    df_filtered = df[~df['symbol'].str.contains("\.|\^")]  # remove weird tickers
    return df_filtered

def df_of_nyse_nasdaq_amex():
    for exchange in EXCHANGE_LIST:
        start_time = time.time()

        df = exchange_to_df(exchange)
        df = df[df['country'] == COUNTRY_USA]  # only US companies
        df = df[df['marketCap'] != '']  # no empty market cap
        df = df[df['marketCap'].astype(float) >= MARKET_CAP_MIN * ONE_MILLION]  # filter by market cap

        print('%s: %.2f seconds' % (exchange, time.time() - start_time))

In [51]:
# start_time = time.time()

# # US companies in NYSE
# df_nyse = exchange_to_df(EXCHANGE_NYSE)
# df_nyse_usa = df_nyse[df_nyse['country'] == COUNTRY_USA]  # only US companies
# df_nyse_usa_mktcap = df_nyse_usa[df_nyse_usa['marketCap'] != '']  # no empty market cap
# df_nyse_usa_mktcap = df_nyse_usa_mktcap[ \
#     df_nyse_usa_mktcap['marketCap'].astype(float) >= MARKET_CAP_MIN * ONE_MILLION]  # filter by market cap

# print('%s: %s seconds' % (EXCHANGE_NYSE, time.time() - start_time))

# df_nyse_usa_mktcap

# US companies in NASDAQ

df_of_nyse_nasdaq_amex()

nyse: 3.12 seconds
nasdaq: 1.26 seconds
amex: 0.84 seconds


In [42]:
df_nyse_usa

,symbol,name,lastsale,netchange,pctchange,marketCap,country,ipoyear,volume,sector,industry,url
0,A,Agilent Technologies Inc. Common Stock,$147.28,0.61,0.416%,44691118178.00,United States,1999,8445127,Capital Goods,Electrical Products,/market-activity/stocks/a
3,AAIC,Arlington Asset Investment Corp Class A (new),$4.00,0.00,0.00%,133924724.00,United States,,4906604,Consumer Services,Real Estate Investment Trusts,/market-activity/stocks/aaic
6,AAN,Aarons Holdings Company Inc. Common Stock,$32.98,-0.18,-0.543%,1185062326.00,United States,2020,1823702,Miscellaneous,Diversified Commercial Services,/market-activity/stocks/aan
7,AAP,Advance Auto Parts Inc Advance Auto Parts Inc W/I,$204.43,3.47,1.727%,13377668194.00,United States,,674967,Consumer Services,Other Specialty Stores,/market-activity/stocks/aap
9,AAT,American Assets Trust Inc. Common Stock,$37.51,0.13,0.348%,2268315448.00,United States,2011,594389,Consumer Services,Real Estate Investment Trusts,/market-activity/stocks/aat
...,...,...,...,...,...,...,...,...,...,...,...,...
3106,YUM,Yum! Brands Inc.,$117.25,0.26,0.222%,34928910189.00,United States,,5229895,Consumer Services,Restaurants,/market-activity/stocks/yum
3107,YUMC,Yum China Holdings Inc. Common Stock,$66.00,-0.98,-1.463%,27759513342.00,United States,2016,2352986,Consumer Services,Restaurants,/market-activity/stocks/yumc
3108,ZBH,Zimmer Biomet Holdings Inc. Common Stock,$165.85,1.46,0.888%,34576756451.00,United States,,1072442,Health Care,Industrial Specialties,/market-activity/stocks/zbh
3119,ZTR,Virtus Total Return Fund Inc.,$9.88,-0.04,-0.403%,243666705.00,United States,1988,113013,Finance,Investment Managers,/market-activity/stocks/ztr


In [13]:
import pandas as pd 
import get_tickers as gt  # https://github.com/shilewenuw/get_all_tickers
from get_tickers import Region
    
if not LOAD_TICKERS_FROM_FILE:
    print('Saving tickers to a CSV file...')
    
    # get tickers (source data are actually from https://www.nasdaq.com/market-activity/stocks/screener)
    tickers = gt.get_tickers()  # tickers of all exchanges: NYSE, NASDAQ, AMEX
#     tickers = gt.get_tickers(AMEX=False)  # tickers from NYSE and NASDAQ only

    # filter tickers
    tickers_filtered = gt.get_tickers_filtered(mktcap_min=MARKET_CAP_MIN)
    
    # save tickers to file
#     gt.save_tickers()  # default filename is tickers.csv
#     gt.save_tickers(filename=FILENAME_TICKERS_ALL)
    
    
    print('Done!')

# else:  # load tickers from file
    
#     df = pd.read_csv(FILENAME_TICKERS_ALL, sep=",")

Saving tickers to a CSV file...
Done!


In [15]:
# ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 46
# tickers_north_america = gt.get_tickers_by_region(Region.NORTH_AMERICA)
# print(tickers_north_america)

# print(len(tickers))
print(len(tickers_filtered))
# print(tickers_filtered)

297
